프로젝트 - Movielens 영화 추천 실습
=======

Matrix Factorization(MF) 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보기

데이터셋은 추천시스템의 MNIST라고 부를만한 Movielens 데이터입니다.     
https://www.kaggle.com/odedgolden/movielens-1m-dataset

- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
- 별점을 시청횟수로 해석해서 생각하겠습니다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

-21.11.02-

# 1. 데이터 준비와 전처리


## 평점 데이터 가져오기 

평점 데이터 : ratings.dat 파일

In [1]:
# 데이터 파일 확인
!more ~/aiffel/Exploration/E9/data/ml-1m/ratings.dat

1::1193::5::978300760
1::661::3::978302109
1::914::3::978301968
1::3408::4::978300275
1::2355::5::978824291
1::1197::3::978302268
1::1287::5::978302039
1::2804::5::978300719
1::594::4::978302268
1::919::4::978301368
1::595::5::978824268
1::938::4::978301752
1::2398::4::978302281
1::2918::4::978302124
1::1035::5::978301753
1::2791::4::978302188
1::2687::3::978824268
1::2018::4::978301777
1::3105::5::978301713
1::2797::4::978302039
1::2321::3::978302205
1::720::3::978300760
1::1270::5::978300055
m--More--(0%)

- Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [2]:
import pandas as pd
import os

rating_file_path=os.getenv('HOME') + '/aiffel/Exploration/E9/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype
---  ------     --------------    -----
 0   user_id    1000209 non-null  int64
 1   movie_id   1000209 non-null  int64
 2   ratings    1000209 non-null  int64
 3   timestamp  1000209 non-null  int64
dtypes: int64(4)
memory usage: 30.5 MB


- 결측치는 없는 것을 확인할 수 있다. 

In [4]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [5]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [6]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'movie_id', 'counts']
ratings = ratings[using_cols]
ratings.head(10)

,user_id,movie_id,counts
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


## 영화 데이터 가져오기 

영화 데이터 : movies.dat 파일

In [7]:
# 데이터 파일 확인
!more ~/aiffel/Exploration/E9/data/ml-1m/movies.dat

1::Toy Story (1995)::Animation|Children's|Comedy
2::Jumanji (1995)::Adventure|Children's|Fantasy
3::Grumpier Old Men (1995)::Comedy|Romance
4::Waiting to Exhale (1995)::Comedy|Drama
5::Father of the Bride Part II (1995)::Comedy
6::Heat (1995)::Action|Crime|Thriller
7::Sabrina (1995)::Comedy|Romance
8::Tom and Huck (1995)::Adventure|Children's
9::Sudden Death (1995)::Action
10::GoldenEye (1995)::Action|Adventure|Thriller
11::American President, The (1995)::Comedy|Drama|Romance
12::Dracula: Dead and Loving It (1995)::Comedy|Horror
13::Balto (1995)::Animation|Children's
14::Nixon (1995)::Drama
15::Cutthroat Island (1995)::Action|Adventure|Romance
16::Casino (1995)::Drama|Thriller
17::Sense and Sensibility (1995)::Drama|Romance
18::Four Rooms (1995)::Thriller
19::Ace Ventura: When Nature Calls (1995)::Comedy
20::Money Train (1995)::Action
21::Get Shorty (1995)::Action|Comedy|Drama
22::Copycat (1995)::Crime|Drama|Thriller
23::Assassins (1995)::Thriller
m--More--(0%)

In [8]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/Exploration/E9/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  3883 non-null   int64 
 1   title     3883 non-null   object
 2   genre     3883 non-null   object
dtypes: int64(1), object(2)
memory usage: 91.1+ KB


- 결측치는 없는 것을 확인할 수 있다.

In [10]:
merged = pd.merge(ratings, movies, on='movie_id')
merged.head()

,user_id,movie_id,counts,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama


# 2. 데이터 분석

추천 모델을 만들기 전에 데이터의 기본적인 정보를 보고 갑니다. 아래 항목들을 확인해 보고 싶습니다.

- ratings에 있는 유니크한 영화 개수
- ratings에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

## ratings에 있는 유니크한 영화 개수

In [11]:
# 영화 수
ratings['movie_id'].nunique()

3628

## ratings에 있는 유니크한 사용자 수

In [12]:
# 사용자 수
ratings['user_id'].nunique()

6039

- pandas.DataFrame.nunique() : 특정 컬럼에 포함된 유니크한 데이터의 개수

## 가장 인기 있는 영화 30개(인기순)

In [13]:
# 인기 많은 영화
movie_count = merged.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30) # 30개 출력

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

# 3. 모델 검증을 위한 사용자 초기 정보 세팅

내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍니다.

In [16]:
# 좋아하는 영화 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주기
my_favorite = [1, 23, 333, 1234, 555]

print([merged[merged['movie_id']==i]['title'].unique()[0] for i in my_favorite])

['Toy Story (1995)', 'Assassins (1995)', 'Tommy Boy (1995)', 'Sting, The (1973)', 'True Romance (1993)']


- movie1 : Toy Story (1995)
- movie23 : Assassins (1995)
- movie333 : Tommy Boy (1995)
- movie1234 : Sting, The (1973)
- movie555 : True Romance (1993)

In [17]:
# '8888'라는 user_id가 위 영화를 봤었다고 가정하겠습니다.
my_mvlist = pd.DataFrame({'user_id': [8888]*5, 'movie_id': my_favorite, 'counts':[4]*5})

if not ratings.isin({'user_id':[8888]})['user_id'].any():  # user_id에 '8888'이라는 데이터가 없다면
    ratings = ratings.append(my_mvlist)                           # 위에 임의로 만든 my_mvlist 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인

,user_id,movie_id,counts
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4
0,8888,1,4
1,8888,23,4
2,8888,333,4
3,8888,1234,4
4,8888,555,4


## 모델에 활용하기 위한 전처리

사람이 태어나면 주민등록번호가, 학교에 가면 출석번호가 있듯이 데이터의 관리를 쉽게 하기 위해 번호를 붙여주고 싶습니다. 우리가 다루는 데이터에서는 user와 movie에 각각에 새로운 번호를 붙이고 싶습니다. 보통 이런 작업을 indexing이라고 합니다. 추천시스템, 자연어처리에서 자주 하는 작업들입니다.

In [18]:
# 제목도 함께 보기 위해 합쳐줌 
merged = pd.merge(ratings, movies, on='movie_id')
merged.head()

,user_id,movie_id,counts,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama


### user_id

In [19]:
# 고유한 유저 찾아내는 코드
user_unique = merged['user_id'].unique()
#print(user_unique)
# 유저 indexing 하는 코드 
user_to_idx = {v:k for k,v in enumerate(user_unique)}

In [20]:
# 인덱싱이 잘 되었는지 확인
print(user_to_idx[8888])

5677


In [21]:
# indexing을 통해 데이터 컬럼 내 값을 다시 바꾸는 코드

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = merged['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(merged):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    merged['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

user_id column indexing OK!!


### movie_id

In [22]:
# 고유한 영화 찾아내는 코드
movie_unique = merged['movie_id'].unique()

# 영화 indexing 하는 코드 
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [23]:
# 인덱싱이 잘 되었는지 확인
print(movie_to_idx[1])

40


In [24]:
# indexing을 통해 데이터 컬럼 내 값을 다시 바꾸는 코드

# movie_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_movie_data = merged['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(merged):   # 모든 row가 정상적으로 인덱싱되었다면
    print('movie_id column indexing OK!!')
    merged['movie_id'] = temp_movie_data   # data['movie_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('movie_id column indexing Fail!!')

movie_id column indexing OK!!


In [25]:
merged.head()

,user_id,movie_id,counts,title,genre
0,0,0,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,0,5,One Flew Over the Cuckoo's Nest (1975),Drama
2,2,0,4,One Flew Over the Cuckoo's Nest (1975),Drama
3,3,0,4,One Flew Over the Cuckoo's Nest (1975),Drama
4,4,0,5,One Flew Over the Cuckoo's Nest (1975),Drama


# 4. CSR(Compressed Sparse Row) matrix 만들기

m명의 사용자들이 n개의 영화에 대해 평가한 데이터를 포함한 (m,n) 사이즈의 평가행렬(Rating Matrix)을 만들 것입니다. 행렬 중 일부는 데이터가 채워져 있지만, 나머지 부분은 데이터가 비어 있습니다.    

추천 시스템의 다양한 모델 중 Matrix Factorization(MF, 행렬분해) 모델을 사용하겠습니다.    
(m,n) 사이즈의 행렬 R을 (m,k) 사이즈의 행렬 P와 (k,n) 사이즈의 행렬 Q로 분해한다면 R이란 그저 P와 Q의 행렬곱으로 표현 가능할 수 있다는 간단한 아이디어입니다.    

유저 X 아이템 평가행렬을 생각해 보면 거대한 행렬을 메모리에 올려놓고 작업한다는 것은 불가능할 것입니다. 이런 경우의 좋은 대안이 되는 것이 CSR(Compressed Sparse Row) Matrix입니다.    
CSR Matrix는 Sparse한 matrix에서 0이 아닌 유효한 데이터로 채워지는 데이터의 값과 좌표 정보만으로 구성하여 메모리 사용량을 최소화하면서도 Sparse한 matrix와 동일한 행렬을 표현할 수 있도록 하는 데이터 구조입니다.

In [26]:
from scipy.sparse import csr_matrix

num_user = merged['user_id'].nunique()
num_movie = merged['movie_id'].nunique()
print(num_user, num_movie)
csr_data = csr_matrix((merged.counts, (merged.user_id, merged.movie_id)), shape= (num_user, num_movie))
csr_data

6040 3628


<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

# 5. MF 모델 학습하기

Matrix Factorization 모델을 implicit 패키지를 사용하여 학습해 봅니다.    
implicit.als 중 AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅니다.
    
Matrix Factorization에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 AlternatingLeastSquares 방식을 사용해 봅니다. 

In [27]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

AlternatingLeastSquares 클래스의 \__init__ 파라미터    
1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지 
2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지 
3. use_gpu : GPU를 사용할 것인지 
4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지     

1,4를 늘릴수록 학습데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야 합니다.

In [60]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=500, regularization=0.01, use_gpu=False, iterations=30, dtype=np.float32)

In [29]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [61]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/30 [00:00<?, ?it/s]

# 6. 선호도 파악

내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 봅니다.

1. 마지막에 추가한 user_id 8888 벡터와 선호하는 영화들(movie_id = [1, 23, 333, 1234, 555])의 벡터 확인
2. 두 벡터를 곱하면 어떤 값이 나오는지

In [62]:
#  user_id 8888 벡터와 movie_id 1 의 벡터 만들기
user8888, movie1 = user_to_idx[8888], movie_to_idx[1]

user8888_vector, movie1_vector = als_model.user_factors[user8888], als_model.item_factors[movie1]

In [63]:
# user_id 8888 벡터 확인 
user8888_vector

array([-0.18062374,  0.04594705, -0.1078848 ,  0.03553069,  0.20162371,
       -0.6712623 ,  0.09027632,  0.03514321, -0.05671945,  0.29449365,
        0.5565499 ,  0.0883837 ,  0.3033283 , -0.20240599,  0.05097035,
        0.09105703,  0.48971316, -0.00735348,  0.00420664,  0.01683307,
       -0.15690911, -0.47401172,  0.15381533, -0.32618332, -0.35564545,
        0.25383502,  0.07692101,  0.01658425,  0.15967937, -0.07926121,
       -0.42363584, -0.00361172, -0.34157512, -0.3026924 ,  0.20664753,
        0.19888069, -0.20299642,  0.01794601,  0.2289282 ,  0.28167862,
        0.13977517, -0.12362383,  0.09195082, -0.3996829 , -0.82339084,
       -0.18338159,  0.4148206 , -0.00930166, -0.00791324, -0.14393446,
        0.14162096, -0.29494265,  0.47155565, -0.05339973,  0.41652504,
       -0.11432562, -0.5762772 , -0.07915893, -0.28202555,  0.22973768,
        0.28233635,  0.10928643, -0.24443032,  0.00811201, -0.33269155,
        0.01550758, -0.0664948 , -0.50057966,  0.0802799 ,  0.33

In [64]:
# movie_id 1 벡터 확인 
movie1_vector

array([-0.02751896, -0.00865638,  0.01535108,  0.00321305,  0.00950127,
       -0.00351146,  0.02642276, -0.01986851,  0.00091059,  0.0083306 ,
       -0.01474267,  0.01239264,  0.00626999,  0.00398124,  0.01525717,
        0.0164193 ,  0.00281313,  0.012031  , -0.00170422, -0.00479028,
        0.00645947, -0.01566906,  0.02002968, -0.00128882, -0.00370597,
        0.00274341, -0.00072956,  0.02362448, -0.01647915, -0.00073943,
        0.01202317,  0.00154696,  0.00638618,  0.03143961,  0.03134539,
        0.00883585,  0.00395351, -0.01026191,  0.00616134,  0.02371066,
        0.02670354, -0.0013579 ,  0.01743193, -0.02798623, -0.03660402,
        0.00732849,  0.0182813 ,  0.01504335,  0.02064517, -0.01791587,
        0.04026437,  0.004452  ,  0.02791325,  0.03589497,  0.01351396,
       -0.01037181, -0.01181616,  0.01427894, -0.0125737 ,  0.02546543,
        0.0188724 ,  0.02834044,  0.00990773, -0.01664959, -0.01176671,
        0.00655285,  0.00595887, -0.00069968,  0.00865488,  0.01

In [53]:
## als_model = AlternatingLeastSquares(factors=100, iterations=15)
# user8888과 movie1을 내적하는 코드 : 선호도 확인
np.dot(user8888_vector, movie1_vector)

0.4327062

In [59]:
## als_model = AlternatingLeastSquares(factors=200, iterations=20)
# user8888과 movie1을 내적하는 코드 : 선호도 확인
np.dot(user8888_vector, movie1_vector)

0.65619546

In [65]:
## als_model = AlternatingLeastSquares(factors=500, iterations=30)
# user8888과 movie1을 내적하는 코드 : 선호도 확인
np.dot(user8888_vector, movie1_vector)

0.88789874

- factors, iterations 을 키워주니 학습이 더 잘 됨을 확인할 수 있습니다.      

가장 학습이 잘된 (factors=500, iterations=30)의 als_model 모델을 사용하도록 하겠습니다. 

In [38]:
#  movie_id = [23, 333, 1234, 555] 에 대한 벡터 만들기
movie23_vector =  als_model.item_factors[movie_to_idx[23]]
movie333_vector = als_model.item_factors[movie_to_idx[333]]
movie1234_vector = als_model.item_factors[movie_to_idx[1234]]
movie555_vector = als_model.item_factors[movie_to_idx[55]]

In [39]:
print(np.dot(user8888_vector, movie1_vector))
print(np.dot(user8888_vector, movie23_vector))
print(np.dot(user8888_vector, movie333_vector))
print(np.dot(user8888_vector, movie1234_vector))
print(np.dot(user8888_vector, movie555_vector))

0.8896343
0.24033943
0.505095
0.78696364
0.03172739


# 7. 비슷한 영화 추천

내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.   
AlternatingLeastSquares 클래스에 구현되어 있는 similar_items 메서드를 통하여 비슷한 영화를 찾습니다.

In [40]:
# 처음으로는 movie1 : Toy Story (1995) 로 찾아보겠습니다.
favorite_movie = 1
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(40, 0.99999976),
 (2938, 0.29779732),
 (3266, 0.28593314),
 (3589, 0.2827054),
 (3260, 0.28150687),
 (2364, 0.28147665),
 (2608, 0.28098834),
 (2598, 0.28040516),
 (2144, 0.27949184),
 (2791, 0.27948555),
 (3475, 0.27924046),
 (3396, 0.27881777),
 (3299, 0.27865252),
 (3624, 0.27859455),
 (3617, 0.27842468)]

- (아티스트의 id, 유사도) Tuple 로 반환하고 있습니다.

In [41]:
# 비슷한 영화 제목, 장르 출력 
[(merged[merged['movie_id']==i[0]]['title'].unique()[0], merged[merged['movie_id']==i[0]]['genre'].unique()[0]) for i in similar_movie]

[('Toy Story (1995)', "Animation|Children's|Comedy"),
 ('Nobody Loves Me (Keiner liebt mich) (1994)', 'Comedy|Drama'),
 ('Amityville: Dollhouse (1996)', 'Horror'),
 ('Soft Toilet Seats (1999)', 'Comedy'),
 ('Price of Glory (2000)', 'Drama'),
 ('Getting Even with Dad (1994)', 'Comedy'),
 ('Wisdom of Crocodiles, The (a.k.a. Immortality) (2000)', 'Romance|Thriller'),
 ('Babyfever (1994)', 'Comedy|Drama'),
 ('Amityville Curse, The (1990)', 'Horror'),
 ('Blood Beach (1981)', 'Action|Horror'),
 ('Walk in the Sun, A (1945)', 'Drama'),
 ('Black Tar Heroin: The Dark End of the Street (1999)', 'Documentary'),
 ('Institute Benjamenta, or This Dream People Call Human Life (1995)',
  'Drama'),
 ('Slaughterhouse (1987)', 'Horror'),
 ('Slappy and the Stinkers (1998)', "Children's|Comedy")]

- 장르를 확인해보니 대부분 Toy Story (1995)의 장르인 comedy를 추천해 주고 있지만 아닌 영화도 많은 것 같다. 

# 8. 유저 맞춤 영화 추천 

내가(user_id : 8888) 가장 좋아할 만한 영화들을 추천받아 봅니다.    
AlternatingLeastSquares 클래스에 구현되어 있는 recommend 메서드를 통하여 제가 좋아할 만한 영화를 추천받습니다.   
filter_already_liked_items 는 유저가 이미 평가한 아이템은 제외하는 Argument입니다.

In [42]:
user = user_to_idx[8888]
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(165, 0.19573247),
 (183, 0.15510938),
 (1353, 0.15058972),
 (127, 0.13944055),
 (1630, 0.1301288),
 (136, 0.12445325),
 (665, 0.122919045),
 (303, 0.12160915),
 (1258, 0.11953376),
 (104, 0.118996166),
 (190, 0.11893334),
 (729, 0.11511405),
 (18, 0.11499144),
 (1471, 0.11288802),
 (1051, 0.11245246),
 (503, 0.11115183),
 (1034, 0.1097365),
 (511, 0.108548164),
 (739, 0.107155815),
 (708, 0.10501094)]

In [46]:
[(merged[merged['movie_id']==i[0]]['title'].unique()[0], merged[merged['movie_id']==i[0]]['genre'].unique()[0]) for i in movie_recommended]

[('Happy Gilmore (1996)', 'Comedy'),
 ('Butch Cassidy and the Sundance Kid (1969)', 'Action|Comedy|Western'),
 ('Network (1976)', 'Comedy|Drama'),
 ('Serpico (1973)', 'Crime|Drama'),
 ('Last Man Standing (1996)', 'Action|Drama|Western'),
 ('Desperado (1995)', 'Action|Romance|Thriller'),
 ('From Russia with Love (1963)', 'Action'),
 ('Dick (1999)', 'Comedy'),
 ('Flatliners (1990)', 'Thriller'),
 ('Like Water for Chocolate (Como agua para chocolate) (1992)',
  'Drama|Romance'),
 ('Monty Python and the Holy Grail (1974)', 'Comedy'),
 ('Dumb & Dumber (1994)', 'Comedy'),
 ('Awakenings (1990)', 'Drama'),
 ('Cinema Paradiso (1988)', 'Comedy|Drama|Romance'),
 ('Mallrats (1995)', 'Comedy'),
 ('Duck Soup (1933)', 'Comedy|War'),
 ('From Dusk Till Dawn (1996)', 'Action|Comedy|Crime|Horror|Thriller'),
 ('Blast from the Past (1999)', 'Comedy|Romance'),
 ('Waterboy, The (1998)', 'Comedy'),
 ('Jerk, The (1979)', 'Comedy')]

In [67]:
# 내가 좋아하는 영화와 장르 확인 
print([(merged[merged['movie_id']==i]['title'].unique()[0], merged[merged['movie_id']==i]['genre'].unique()[0]) for i in my_favorite])

[('James and the Giant Peach (1996)', "Animation|Children's|Musical"), ("Schindler's List (1993)", 'Drama|War'), ('Jeremiah Johnson (1972)', 'Western'), ('Dave (1993)', 'Comedy|Romance'), ('Heart and Souls (1993)', 'Comedy|Fantasy')]


## 기여도 확인

als_model.explain()은 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도(합이 콘텐츠의 점수)를 반환합니다. 어떤 영화들이 이 추천에 얼마나 기여하고 있는 지 확인해 봅니다. 

In [44]:
# 처음으로 나온 Happy Gilmore (1996) 확인
recommand_mv = merged[merged['title']=='Happy Gilmore (1996)']['movie_id'].unique()[0]
explain = als_model.explain(user, csr_data, itemid=recommand_mv)

In [68]:
[(merged[merged['movie_id']==i[0]]['title'].unique()[0], merged[merged['movie_id']==i[0]]['genre'].unique()[0], i[1]) for i in explain[1]]

[('Tommy Boy (1995)', 'Comedy', 0.11386267126224645),
 ('Sting, The (1973)', 'Comedy|Crime', 0.03167182437279031),
 ('True Romance (1993)', 'Action|Crime|Romance', 0.028569007277977312),
 ('Assassins (1995)', 'Thriller', 0.020850113501616474),
 ('Toy Story (1995)', "Animation|Children's|Comedy", -0.0005416760877645691)]

- ['Happy Gilmore (1996)', 'Comedy'] 는      
    ['Tommy Boy (1995)', 'Comedy'] 와 ['Sting, The (1973)', 'Comedy|Crime']의 영향을 많이 받아 comedy 장르의 영화를 추천 받은것 같다. 

# 회고 

1. CSR matrix가 정상적으로 만들어졌다.     
    사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.   
    
2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.     
    사용자와 아이템 벡터 내적수치가 의미있게 형성된 것 같다.      
    하지만 내적 수치의 정확한 의미를 더 공부할 것이다.      
    
3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.     
    MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.     
    하지만 데이터에 어떤 영화가 포함되어 있는지 잘 파악하지 못하여 임의의 영화로 추천 시스템을 만들어 보아 영화에 대한 정보는 정확하게 파악하기가 어려웠다. 